In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


pd.options.display.float_format = '{:20.2f}'.format


('display.max_columns', 999)


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
df = pd.read_excel("/home/fdr/Downloads/dataset/online_retail_II.xlsx")
pd.head(10)